In [1]:
import numpy as np
import math
import os

In [2]:
from sklearn import mixture

In [7]:
# load phone list
phone_path = "C:\\Users\\FG\\Desktop\\PhD\\Research\\Reservoirs\\datasets\\TIMIT-MFCCs\\TIMIT_phone_list.txt"
phone_path = "/home/felix/Research/Reservoirs/datasets/TIMIT_phone_list.txt"
phone_list = load_phone_file(phone_path)
print phone_list

['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']


In [8]:
#load mfccs into sklearn observations, each frame is an obs
# (CURRENTLY FALSE: shape should be nb_files -> nb_frames -> 39 MFFC dim)

train_TIMIT_dir = "C:\\Users\\FG\Desktop\\PhD\\Research\\Reservoirs\\datasets\\TIMIT-MFCCs\\test"
train_TIMIT_dir = "/home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test"

train_obs = []
train_obs_labels = []

# walk the directories
for (path, dirs, files) in os.walk(train_TIMIT_dir):
    for file in files:
        # skip the SA files
        #dev, only work on file si1573.mfcc.csv     "si1573" in file and
        if ".mfcc" in file  and "sa" not in file:
            #check if corresponding .phn file exists
            if not os.path.exists(path + "/" + file[:-8] + "phn"):
                print path + "/" + file[:-8] + "phn"
                print "corresponding .phn file does not exist!"
            else:
                
                print "working on: " + file
                print "from path : " + path

                # open the files
                mfcc_file = open(path + "/" + file)
                phn_file = open(path + "/" + file[:-8] + "phn")

                # extract phone times
                phone_times = []
                for phn_line in phn_file:
                    phone_times.append(phn_line.split())
                # transpose for easier use
                phone_times = map(list, zip(*phone_times))

                # skip mfcc_file header
                next(mfcc_file)

                # reset frame count
                frame_cnt = 0

                # for each line of mfcc_file
                for mfcc_line in mfcc_file:

                    # increment frame count
                    frame_cnt += 1 

                    # print "frame line #:", frame_cnt 

                    # frame start time in seconds
                    start_t = mfcc_line.split(";")[1]

                    # create frame (skiping first 2 values, frame_index and frame_time)
                    frame = map( float,  mfcc_line.split(";")[2:])
                    # print numpy.shape(frame)
                    # print frame

                    # find correspond phoneme and index in the list
                    phn_index = find_phn_index(start_t, phone_times, phone_list)

                    # add to instances
                    train_obs.append(frame)
                    train_obs_labels.append(phone_list[phn_index])

working on: sx12.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: sx192.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: sx282.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: si1362.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: si732.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: sx102.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: sx372.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: si1992.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mmab0
working on: si1720.mfcc.csv
from path : /home/felix/Research/Reservoirs/datasets/TIMIT-MFCCs/test/dr3/mhpg0
working on: si1090.mfcc.csv
from pa

In [9]:
print np.shape(train_obs)
print np.shape(train_obs_labels)

(410920, 39)
(410920,)


In [10]:
print train_obs[0]
print train_obs_labels[0:100]

[-17.42208, -17.85209, -29.4464, -16.99902, -10.496, -9.797142, -11.33554, 7.052387, 0.90574, -6.057405, -10.80134, 1.023798, -17.53597, -0.3289059, 2.153232, 0.7268575, 0.5420668, 1.733999, 1.87351, 0.5596565, -4.216178, -0.3989117, 3.487705, 1.493496, -0.9547125, -0.2499979, -0.7495419, 0.3893737, 0.5015411, 0.3174773, -0.4864308, 0.6648478, 0.04972706, -0.2317928, -0.3564923, -0.1835683, 1.234655, 0.02363437, -0.04550938]
['h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'h#', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'w', 'ih', 'ih', 'ih', 'ih', 'ih', 'ih', 'l', 'l', 'l', 'l', 'l', 'l', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'bcl', 'bcl', 'bcl', 'b', 'ih', 'ih', 'ih', 'ih', 'ih', 'ih', 'n', 'n', 'n', 'n', 'n', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'pau', 'w', 'w', 'w', 'w', 'eh', 'eh', 'eh', 'eh', 'eh', 'eh', 'eh', 'eh', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'q', 'q', 'q', 'q', 'q', 'q', 'q']


In [11]:
#create gmm
n_components = 10
g = mixture.GMM(n_components)

In [14]:
# fit the GMM to the observations
g.fit(train_obs) 

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=10, n_init=1, n_iter=100, params='wmc', random_state=None,
  thresh=0.01)

In [ ]:
# pred = g.predict(train_obs)
# print train_obs_labels
# print pred

In [18]:
print np.round(g.score(train_obs).mean(), 2)
print np.round(g.score(train_obs).var(), 2)
print np.round(g.score(train_obs)[0:5] )

-104.47
121.88
[ -82.  -91.  -95. -100.  -90.]


In [5]:
def load_phone_file(phonePath):
    phones_list = []
    try:
        f = open(phonePath)
        for line in f:
            phones_list.append(line.split())
        # close f
        f.close()

    except IOError:
        print "missing phone file"

    # flatten the list using nested list comprehension
    return [val for subl in phones_list for val in subl]

In [4]:
def find_phn_index(start_time, phone_times, phone_list):
    # given a frame start time, a list of phone times for this audio file, and a list of phones
    # returns the list of the phone corresponding to this frame in the list

    sample_rate = 16000.0 #!!!

    for i in range( len(phone_times[0])):

        # print phone_times[0][i], float(start_time)*sample_rate, phone_times[1][i]

        if float(phone_times[0][i]) <= float(start_time)*sample_rate <= float(phone_times[1][i]):
            return phone_list.index(phone_times[2][i])

    # if nothing matches, return silence
    # this happens when the start time is beyond the last frame
    # see timit doc
    return phone_list.index("h#")